# Creating a Turboprop Powered Aircraft Example

Here we will create a more advanced `EngineModel`, the `TurbopropModel`. This is a complicated type of engine that combines several different models into a single unit. The `TurbopropModel` has three individual pieces needed to create it - a "shaft power model" that describes how the turboprop turns energy into shaft power, such as a turboshaft or motor, a "gearbox model" for adjusting the shaft RPM, and a "propeller model" to convert shaft power into thrust. Let's walk through two examples: first building a turboprop with the minimum amount of required information, and then a second example where we replace individual models in the `TurbopropModel` with custom ones of our own choice.

In [ ]:
import openmdao.api as om
import aviary.api as av
import numpy as np

Aircraft = av.Aircraft
Mission = av.Mission
Dynamic = av.Dynamic

# define the minimum option set for a turboprop
options = av.AviaryValues()
options.set_val(av.Settings.VERBOSITY, 0) # quiet unneeded printouts
options.set_val(Aircraft.Engine.FIXED_RPM, 1500, units='rpm')

# EngineDeck minimum option set
options.set_val(
    Aircraft.Engine.DATA_FILE,
    av.get_path('models/engines/turboshaft_1120hp.deck')
)
options.set_val(Aircraft.Engine.INTERPOLATION_METHOD, 'slinear')
options.set_val(Aircraft.Engine.GENERATE_FLIGHT_IDLE, False)
options.set_val(Aircraft.Engine.IGNORE_NEGATIVE_THRUST, False)
options.set_val(Aircraft.Engine.SUBSONIC_FUEL_FLOW_SCALER, 1.0)
options.set_val(Aircraft.Engine.SUPERSONIC_FUEL_FLOW_SCALER, 1.0)
options.set_val(Aircraft.Engine.FUEL_FLOW_SCALER_CONSTANT_TERM, 0.0)
options.set_val(Aircraft.Engine.FUEL_FLOW_SCALER_LINEAR_TERM, 1.0)
options.set_val(Aircraft.Engine.CONSTANT_FUEL_CONSUMPTION, 0.0, units='lbm/h')

# Hamilton Standard propeller minimum option set
options.set_val(Aircraft.Engine.Propeller.NUM_BLADES, val=4, units='unitless')
options.set_val(Aircraft.Engine.Propeller.COMPUTE_INSTALLATION_LOSS, True)

# Initialize turboprop model. Model uses an EngineDeck built from `options`, basic 
# gearbox model with default efficiency of 1, and the Hamilton Standard propeller model
# "turboprop" is ready to be included in an AviaryProblem
turboprop = av.TurbopropModel(name='turboprop_example', options=options)

# build an openmdao problem to demonstrate running the turboprop model in isolation
prob = om.Problem()

# define some example flight conditions mimicking idle, SLS, and TOC
machs = [0, 0, 0.6]
altitudes = [0, 0, 25000]
throttles = [0, 1, 1]

flight_conditions = om.IndepVarComp(
    Dynamic.Atmosphere.MACH, np.array(machs),
    units='unitless'
)
flight_conditions.add_output(
    Dynamic.Mission.ALTITUDE,
    np.array(altitudes),
    units='ft'
)
flight_conditions.add_output(
    Dynamic.Vehicle.Propulsion.THROTTLE,
    np.array(throttles),
    units='unitless'
)

prob.model.add_subsystem(
    'flight_conditions',
    flight_conditions,
    promotes=['*']
)

# calculate atmospheric properties
prob.model.add_subsystem(
    name='atmosphere',
    subsys=av.Atmosphere(num_nodes=3, input_speed_type=av.SpeedType.MACH),
    promotes=['*'],
)

# add the turboprop mission components
prob.model.add_subsystem(
    'turboprop_mission',
    turboprop.build_mission(num_nodes=3, aviary_inputs=options),
    promotes=['*']
)

prob.setup(force_alloc_complex=False)
prob.set_val(Aircraft.Nacelle.AVG_DIAMETER, 1.0, 'ft')
prob.set_val(Aircraft.Engine.Propeller.ACTIVITY_FACTOR, 100, units="unitless")
prob.set_val(Aircraft.Engine.Propeller.INTEGRATED_LIFT_COEFFICIENT, 0.5, units="unitless")
prob.set_val(Aircraft.Engine.Propeller.TIP_SPEED_MAX, 800, units="ft/s")
prob.set_val(Aircraft.Engine.Propeller.DIAMETER, 10, 'ft')

prob.run_model()

thrust = prob.get_val(Dynamic.Vehicle.Propulsion.THRUST, units='lbf')
fuel_flow = prob.get_val(
    Dynamic.Vehicle.Propulsion.FUEL_FLOW_RATE_NEGATIVE, units='lbm/h'
)

print(f"Thrust Produced (lbf) = {thrust}")
print(f"Fuel Flow (lbm/hr) = {fuel_flow}")


We can see the turboprop successfully run our simulated mission. Next we will replace the shaft power model, currently an `EngineDeck` representing a turboshaft, to a motor model. This will turn our turboprop into an electroprop. Replacing sub-models in the turboprop is simple, we must first define our motor model, and then provide it to the `TurbopropModel`.

In [ ]:
# define the minimum option set for an all-electric "electroprop"
options = av.AviaryValues()
options.set_val(Aircraft.Engine.FIXED_RPM, 1500, units='rpm')

# Hamilton Standard propeller minimum option set
options.set_val(Aircraft.Engine.Propeller.NUM_BLADES, val=4, units='unitless')
options.set_val(Aircraft.Engine.Propeller.COMPUTE_INSTALLATION_LOSS, True)

# Create the motor model
motor_model = av.MotorBuilder()

# Initialize turboprop model. Model uses the provided motor model, basic gearbox model
# with default efficiency of 1, and the Hamilton Standard propeller model
# "electroprop" is ready to be included in an AviaryProblem
electroprop = av.TurbopropModel(
    name='turboprop_example',
    options=options,
    shaft_power_model=motor_model
)

# build an openmdao problem to demonstrate running the electroprop model in isolation
prob = om.Problem()

# define some example flight conditions mimicking idle, SLS, and TOC
machs = [0, 0, 0.6]
altitudes = [0, 0, 25000]
throttles = [0, 1, 1]

flight_conditions = om.IndepVarComp(
    Dynamic.Atmosphere.MACH, np.array(machs),
    units='unitless'
)
flight_conditions.add_output(
    Dynamic.Mission.ALTITUDE,
    np.array(altitudes),
    units='ft'
)
flight_conditions.add_output(
    Dynamic.Vehicle.Propulsion.THROTTLE,
    np.array(throttles),
    units='unitless'
)

prob.model.add_subsystem(
    'flight_conditions',
    flight_conditions,
    promotes=['*']
)

# calculate atmospheric properties
prob.model.add_subsystem(
    name='atmosphere',
    subsys=av.Atmosphere(num_nodes=3, input_speed_type=av.SpeedType.MACH),
    promotes=['*'],
)

# add the turboprop mission components
prob.model.add_subsystem(
    'turboprop_mission',
    turboprop.build_mission(num_nodes=3, aviary_inputs=options),
    promotes=['*']
)

prob.setup(force_alloc_complex=False)
prob.set_val(Aircraft.Nacelle.AVG_DIAMETER, 1.0, 'ft')
prob.set_val(Aircraft.Engine.Propeller.ACTIVITY_FACTOR, 100, units="unitless")
prob.set_val(Aircraft.Engine.Propeller.INTEGRATED_LIFT_COEFFICIENT, 0.5, units="unitless")
prob.set_val(Aircraft.Engine.Propeller.TIP_SPEED_MAX, 800, units="ft/s")
prob.set_val(Aircraft.Engine.Propeller.DIAMETER, 10, 'ft')

prob.run_model()

thrust = prob.get_val(Dynamic.Vehicle.Propulsion.THRUST, units='lbf')
fuel_flow = prob.get_val(
    Dynamic.Vehicle.Propulsion.ELECTRIC_POWER_IN_TOTAL, units='kW'
)

print(f"Thrust Produced (lbf) = {thrust}")
print(f"Electric Power (kW) = {fuel_flow}")